Hardcoded to domain_id=='Condition', can be extended for other NERs

In [0]:
%python
dbutils.widgets.text("EXPERIMENT_ID","4090675059920416")
dbutils.widgets.text("CATALOG","edav_prd_cdh")
dbutils.widgets.text("SRC_SCHEMA","cdh_abfm_phi_ra")
dbutils.widgets.text("DEST_SCHEMA","cdh_abfm_phi_exploratory")
dbutils.widgets.text("NLP_TABLE","ml_note_random_nlp")
# {CATALOG}.{SRC_SCHEMA}.{table)
# {CATALOG}.{DEST_SCHEMA}.{table)

CREATE WIDGET TEXT CATALOG DEFAULT "edav_dev_prd";
CREATE WIDGET TEXT SRC_SCHEMA DEFAULT "cdh_abfm_phi_ra";
CREATE WIDGET TEXT DEST_SCHEMA DEFAULT "cdh_abfm_phi_exploratory";

In [0]:
%python
# 15.2 and above PARMS=dbutils.widgets.getAll()
# 15.2 and above print(PARMS)
EXPERIMENT_ID=dbutils.widgets.get("EXPERIMENT_ID")
CATALOG=dbutils.widgets.get("CATALOG").strip()
SRC_SCHEMA=(dbutils.widgets.get("SRC_SCHEMA")).strip()
DEST_SCHEMA=(dbutils.widgets.get("DEST_SCHEMA")).strip()
NLP_TABLE=(dbutils.widgets.get("NLP_TABLE")).strip()

In [0]:
%python
sqltext=f"create or replace table {CATALOG}.{DEST_SCHEMA}.fact_person_summary as "\
"select person_id,observation_datetime,domain_id, count(distinct concept_code) as codes "\
f" from {CATALOG}.{SRC_SCHEMA}.fact_person "\
" where observation_datetime is not null"\
" group by person_id, observation_datetime,domain_id"
results=spark.sql(sqltext)
display(results)

In [0]:
%python
sqltext=f"create or replace table {CATALOG}.{DEST_SCHEMA}.{NLP_TABLE}_entities as "\
"select note_id,count(nlp_category) as ner_count from "\
"(select  note_id, snippet, nlp_category, count(score) as counts "\
f" from  {CATALOG}.{DEST_SCHEMA}.{NLP_TABLE} group by  note_id, snippet, nlp_category ) "\
"group by note_id "
print(sqltext)
results=spark.sql(sqltext)
display(results)

In [0]:
%python
sqltext=f"create or replace table {CATALOG}.{DEST_SCHEMA}.{NLP_TABLE}_details as "\
"select note_id,"\
"person_id,"\
f"note_datetime from  {CATALOG}.{SRC_SCHEMA}.note n "\
     f"where note_id in (select distinct e.note_id from  {CATALOG}.{DEST_SCHEMA}.{NLP_TABLE}_entities e)"
print(sqltext)
results=spark.sql(sqltext)
display(results)

In [0]:
sqltext=f"create or replace table {CATALOG}.{DEST_SCHEMA}.{NLP_TABLE}_okr_summary as "\
"select d.person_id,d.note_datetime,count(e.note_id) as note_count, sum(e.ner_count) as total_ner " \
f"from {CATALOG}.{DEST_SCHEMA}.{NLP_TABLE}_details d "\
f"left join {CATALOG}.{DEST_SCHEMA}.{NLP_TABLE}_entities e "\
"on d.note_id=e.note_id "\
"group by d.person_id, d.note_datetime"
print(sqltext)
results=spark.sql(sqltext)
display(results)


In [0]:
sqltext=f"create or replace table {CATALOG}.{DEST_SCHEMA}.{NLP_TABLE}_condition_okr as "\
"select f.person_id,f.observation_datetime,f.codes,n.total_ner,n.note_count,CAST(n.note_count>f.codes  as Integer) as okr_val "\
f"from {CATALOG}.{DEST_SCHEMA}.fact_person_summary f "\
f"join {CATALOG}.{DEST_SCHEMA}.{NLP_TABLE}_okr_summary n "\
"on f.person_id=n.person_id and f.observation_datetime=n.note_datetime "\
"where f.domain_id like '%Condition%' "
print(sqltext)
results=spark.sql(sqltext)
display(results)

In [0]:
%python
okr_summary=spark.sql(f"select count(*) as records,sum(okr_val) as OKR2 from {CATALOG}.{DEST_SCHEMA}.{NLP_TABLE}_condition_okr") 
display(okr_summary)

In [0]:
%python
import mlflow
records,ners=okr_summary.toPandas().values[0]
okr=ners/records
with mlflow.start_run(experiment_id=EXPERIMENT_ID):
    mlflow.log_metric(f"efficiency_percentage", okr)
    mlflow.log_metric(f"records",records)